# Example of using the PyRadiomics toolbox in Python

First, import some built-in Python modules needed to get our testing data.
Second, import the toolbox, only the `featureextractor` is needed, this module handles the interaction with other parts of the toolbox.

In [1]:
from __future__ import print_function
import six
import os  # needed navigate the system to get the input data

import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics

### Setting up data

Test cases can be downloaded to temporary files. This is handled by the `radiomics.getTestCase()` function, which checks if the requested test case is available and if not, downloads it. It returns a tuple with the location of the image and mask of the requested test case, or (None, None) if it fails.

Alternatively, if the data is available somewhere locally, this directory can be passed as a second argument to `radiomics.getTestCase()`. If that directory does not exist or does not contain the testcase, functionality reverts to default and tries to download the test data.

If getting the test case fails, PyRadiomics will log an error explaining the cause.

In [2]:
# Get the testCase
imagePath, maskPath = radiomics.getTestCase('brain1')

if imagePath is None or maskPath is None:  # Something went wrong, in this case PyRadiomics will also log an error
    raise Exception('Error getting testcase!')  # Raise exception to prevent cells below from running in case of "run all"

# Additonally, store the location of the example parameter file, stored in \pyradiomics\examples/exampleSettings
paramPath = os.path.join('..', 'examples', 'exampleSettings', 'Params.yaml')
print('Parameter file, absolute path:', os.path.abspath(paramPath))

Parameter file, absolute path: E:\Git-Repos\pyradiomics\examples\exampleSettings\Params.yaml


### Instantiating the extractor

Now that we have our input, we need to define the parameters and instantiate the extractor.
For this there are three possibilities:

1. Use defaults, don't define custom settings

2. Define parameters in a dictionary, control filters and features after initialisation

3. Use a parameter file

##### Method 1, use defaults

In [3]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeaturesExtractor()

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor._enabledImagetypes)
print('Enabled features:\n\t', extractor._enabledFeatures)

Extraction parameters:
	 {'normalize': False, 'distances': [1], 'enableCExtensions': True, 'removeOutliers': None, 'minimumROISize': None, 'force2Ddimension': 0, 'label': 1, 'padDistance': 5, 'force2D': False, 'interpolator': 'sitkBSpline', 'preCrop': False, 'resampledPixelSpacing': None, 'resegmentRange': None, 'minimumROIDimensions': 1, 'additionalInfo': True, 'normalizeScale': 1}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'gldm': [], 'glcm': [], 'shape': [], 'firstorder': [], 'glrlm': [], 'glszm': [], 'ngtdm': []}


##### Method 2, hard-coded settings:

In [4]:
# First define the settings
settings = {}
settings['binWidth'] = 20
settings['sigma'] = [1, 2, 3]

# Instantiate the extractor
extractor = featureextractor.RadiomicsFeaturesExtractor(**settings)  # ** 'unpacks' the dictionary in the function call

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor._enabledImagetypes)  # Still the default parameters
print('Enabled features:\n\t', extractor._enabledFeatures)  # Still the default parameters

Extraction parameters:
	 {'normalize': False, 'distances': [1], 'enableCExtensions': True, 'removeOutliers': None, 'minimumROISize': None, 'binWidth': 20, 'force2Ddimension': 0, 'label': 1, 'padDistance': 5, 'force2D': False, 'interpolator': 'sitkBSpline', 'preCrop': False, 'resampledPixelSpacing': None, 'resegmentRange': None, 'sigma': [1, 2, 3], 'minimumROIDimensions': 1, 'additionalInfo': True, 'normalizeScale': 1}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'gldm': [], 'glcm': [], 'shape': [], 'firstorder': [], 'glrlm': [], 'glszm': [], 'ngtdm': []}


In [5]:
# This cell is equivalent to the previous cell
extractor = featureextractor.RadiomicsFeaturesExtractor(binWidth=20, sigma=[1, 2, 3])  # Equivalent of code above

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor._enabledImagetypes)  # Still the default parameters
print('Enabled features:\n\t', extractor._enabledFeatures)  # Still the default parameters

Extraction parameters:
	 {'normalize': False, 'distances': [1], 'enableCExtensions': True, 'removeOutliers': None, 'minimumROISize': None, 'binWidth': 20, 'force2Ddimension': 0, 'label': 1, 'padDistance': 5, 'force2D': False, 'interpolator': 'sitkBSpline', 'preCrop': False, 'resampledPixelSpacing': None, 'resegmentRange': None, 'sigma': [1, 2, 3], 'minimumROIDimensions': 1, 'additionalInfo': True, 'normalizeScale': 1}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'gldm': [], 'glcm': [], 'shape': [], 'firstorder': [], 'glrlm': [], 'glszm': [], 'ngtdm': []}


In [6]:
# Enable a filter (in addition to the 'Original' filter already enabled)
extractor.enableImageTypeByName('LoG')
print('')
print('Enabled filters:\n\t', extractor._enabledImagetypes)

# Disable all feature classes, save firstorder
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('firstorder')
print('')
print('Enabled features:\n\t', extractor._enabledFeatures)

# Specify some additional features in the GLCM feature class
extractor.enableFeaturesByName(glcm=['Autocorrelation', 'Homogeneity1', 'SumSquares'])
print('')
print('Enabled features:\n\t', extractor._enabledFeatures)


Enabled filters:
	 {'Original': {}, 'LoG': {}}

Enabled features:
	 {'firstorder': []}

Enabled features:
	 {'firstorder': [], 'glcm': ['Autocorrelation', 'Homogeneity1', 'SumSquares']}


##### Method 3, using a parameter file

In [7]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeaturesExtractor(paramPath)

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor._enabledImagetypes)
print('Enabled features:\n\t', extractor._enabledFeatures)

Extraction parameters:
	 {'normalize': False, 'distances': [1], 'enableCExtensions': True, 'removeOutliers': None, 'minimumROISize': None, 'binWidth': 25, 'force2Ddimension': 0, 'label': 1, 'padDistance': 5, 'force2D': False, 'weightingNorm': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'resampledPixelSpacing': None, 'resegmentRange': None, 'minimumROIDimensions': 1, 'additionalInfo': True, 'normalizeScale': 1}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'glcm': ['Autocorrelation', 'JointAverage', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'JointEnergy', 'JointEntropy', 'Imc1', 'Imc2', 'Idm', 'Idmn', 'Id', 'Idn', 'InverseVariance', 'MaximumProbability', 'SumEntropy', 'SumSquares'], 'shape': None, 'gldm': None, 'firstorder': [], 'glrlm': None, 'glszm': None}


### Extract features

Now that we have our extractor set up with the correct parameters, we can start extracting features:

In [8]:
result = extractor.execute(imagePath, maskPath)

In [9]:
print('Result type:', type(result))  # result is returned in a Python ordered dictionary)
print('')
print('Calculated features')
for key, value in six.iteritems(result):
    print('\t', key, ':', value)

Result type: <class 'collections.OrderedDict'>

Calculated features
	 general_info_BoundingBox : (162, 84, 11, 47, 70, 7)
	 general_info_EnabledImageTypes : {'Original': {}}
	 general_info_GeneralSettings : {'normalize': False, 'distances': [1], 'enableCExtensions': True, 'removeOutliers': None, 'minimumROISize': None, 'binWidth': 25, 'force2Ddimension': 0, 'label': 1, 'padDistance': 5, 'force2D': False, 'weightingNorm': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'resampledPixelSpacing': None, 'resegmentRange': None, 'minimumROIDimensions': 1, 'additionalInfo': True, 'normalizeScale': 1}
	 general_info_ImageHash : 5c9ce3ca174f0f8324aa4d277e0fef82dc5ac566
	 general_info_ImageSpacing : (0.7812499999999999, 0.7812499999999999, 6.499999999999998)
	 general_info_MaskHash : 9dc2c3137b31fd872997d92c9a92d5178126d9d3
	 general_info_NumpyVersion : 1.14.0
	 general_info_PyWaveletVersion : 0.5.2
	 general_info_SimpleITKVersion : 0.9.1
	 general_info_Version : 1.3.0.post54.dev0+gc4759a4